In [1]:
# Install required libraries
!pip install transformers datasets peft accelerate evaluate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#Install and use wandb for performing a Hyper-parameter sweep
!pip install -U wandb --quiet          # if not already installed
import wandb
wandb.login()                          # paste the API key when asked

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 45.1 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yaagni-raolji (yaagni-raolji-schbangq) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [5]:
import pandas as pd
from datasets import Dataset
df = pd.read_json("hr_sentiment_starter.jsonl", lines=True)
dataset = Dataset.from_pandas(df)
label_map = {
    "highly engaged": 0,
    "content": 1,
    "disengaged": 2,
    "at risk of leaving": 3
}

dataset = dataset.map(lambda x: {"label": label_map[x["label"]]})

dataset = dataset.train_test_split(test_size=0.2)
dataset['validation'] = dataset.pop('test')

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [7]:
# Initialize model
from transformers import AutoModelForSequenceClassification
model_name="j-hartmann/emotion-english-distilroberta-base"

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=4,
        ignore_mismatched_sizes=True
    )

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
# Tokenize dataset
def tokenize(example):
    return tokenizer(example['pros_cons'], padding='max_length', truncation=True)

tokenized = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

In [9]:
sweep_config = {
    "method": "grid",          # or "random", "bayes"
    "metric": {
        "name": "eval_f1",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate":  {"values": [2e-5, 5e-5]},
        "per_device_train_batch_size": {"values": [8,16]},
        "num_train_epochs": {"values": [3,4,5]}
    }
}
sweep_id = wandb.sweep(sweep_config, project="hr-sentiment-sweep-v2")


Create sweep with ID: ril6u5ce
Sweep URL: https://wandb.ai/yaagni-raolji-schbangq/hr-sentiment-sweep-v2/sweeps/ril6u5ce


In [19]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

def sweep_train():
    # Fetch current sweep hyper-params from wandb.config
    run   = wandb.init()            # project/name is inherited from the sweep
    config = run.config

    args = TrainingArguments(
        output_dir="./results/sentiment-model",
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        greater_is_better=True,
        learning_rate=config.learning_rate,
        per_device_train_batch_size=config.per_device_train_batch_size,
        per_device_eval_batch_size=config.per_device_train_batch_size,
        num_train_epochs=config.num_train_epochs,
        report_to="wandb",
        run_name=run.name
    )

    trainer = Trainer(
        model_init=model_init,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    # Save model after training
    model_dir = f"./results/model-{run.id}"
    trainer.save_model(model_dir)

    # At the end, log classification report on validation set
    preds_output = trainer.predict(tokenized["validation"])
    preds  = preds_output.predictions.argmax(-1)
    labels = preds_output.label_ids

    from sklearn.metrics import classification_report
    report = classification_report(
        labels, preds,
        target_names=list(label_map.values()),
        output_dict=True
    )
    wandb.log({"classification_report": report})
    # Log the artifact
    artifact = wandb.Artifact(f"model-{wandb.run.name}", type="model")
    artifact.add_dir("./results/sentiment-model")  # This is where Trainer saved the model
    wandb.log_artifact(artifact)
    run.finish()


In [21]:
wandb.agent(sweep_id, function=sweep_train)

wandb: Agent Starting Run: e6uucqtn with config:
wandb: 	learning_rate: 5e-05
wandb: 	num_train_epochs: 3
wandb: 	per_device_train_batch_size: 8


<ipython-input-19-d5bca177abf3>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

...

<ipython-input-19-d5bca177abf3>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found 

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.569167,0.800000,0.801697
2,No log,0.484194,0.812500,0.813727
3,No log,0.472330,0.831250,0.832308
4,No log,0.448282,0.850000,0.850005
5,No log,0.449279,0.850000,0.849799


wandb: Adding directory to artifact (./results/sentiment-model)... Done. 94.7s


eval/accuracy,▁▃▅██
eval/f1,▁▃▅██
eval/loss,█▃▂▁▁
eval/runtime,▁▅▇█▃
eval/samples_per_second,█▄▂▁▆
eval/steps_per_second,█▄▂▁▆
test/accuracy,▁
test/f1,▁
test/loss,▁
test/runtime,▁
test/samples_per_second,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [24]:
# Use this if you want to load the model from artifact
import wandb

# Log in first
wandb.login()

# Restore the best model from W&B
run = wandb.init(project="hr-sentiment-sweep-v2", id="q8smpdk6", resume="allow")

artifact = run.use_artifact("model-worthy-sweep-11:v0")  # or "model:v0" if named that way
model_dir = artifact.download()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


KeyboardInterrupt: 

In [27]:
# Use this if you want to load the model from local
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_dir = "./results/model-q8smpdk6"
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)


In [39]:
from transformers import DataCollatorWithPadding
import pandas as pd
from datasets import Dataset

df = pd.read_json("employee_reviews_testset_100_each.jsonl", lines=True)
test_dataset = Dataset.from_pandas(df)

test_dataset = test_dataset.map(lambda x: {"label": label_map[x["label"]]})

test_dataset = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [38]:
from transformers import Trainer

trainer = Trainer(model=model, compute_metrics=compute_metrics)
results = trainer.evaluate(test_dataset)
print(results)


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).


{'eval_loss': 0.3804772198200226, 'eval_model_preparation_time': 0.0015, 'eval_accuracy': 0.9175, 'eval_f1': 0.9169230320720958, 'eval_runtime': 5.283, 'eval_samples_per_second': 75.714, 'eval_steps_per_second': 9.464}
